# Print Output Table From Bolocalc

This notebook, once pointed to an experiment directory, prints out a variety of relevant parameters (optical loading, NEPs, NETs) for each channel.  (For runs after re-org 1/25/2021) 

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import sys
import toml
#import os

In [2]:
# Import unpack.py from the BoloCalc/src/ directory
BoloCalcPath = 'BoloCalc/'
src_path = BoloCalcPath+'src'
if src_path not in sys.path:
    sys.path.append(src_path)

import unpack as up
# Instantiate an Unpack object
unpack = up.Unpack()   

# BoloCalc's "unpack" python interface does not help us access the Psats or
# safety factor, so we define it here by hand.  Change this if your simulation differs.
f_safety = 3.0

# Define which bolocalc simulation you're going to look at.
The new reorganization has only one telescope at one site (but with multiple tubes and channels per tube).

In [3]:
#Atacama_CDLAT_RealBands_20210202
#Atacama_SAT_MixedBands_20210115
#Pole_SAT_BicepLike_20210115
#Pole_SAT_FlatBands_20210115
#Pole_TMALAT_FlatBands_20210202
#Pole_TMALAT_RealBands_20210202
#Pole_SAT_Bicep3bands_20210130
run = 'Atacama_CDLAT_FlatBands_20210203'

exp_dir = BoloCalcPath+'Experiments/CMB-S4/' + run + '/'

In [4]:
# Load in the two dictionaries that contain all the info we need.
#unpack.unpack_sensitivities(exp_dir)
#unpack.unpack_optical_powers(exp_dir)
#telescope = list(unpack.sens_outputs[exp].keys())[0]  # We are working with runs that only have one telescope.
#print(telescope)

#bands = list(unpack.sens_outputs[run][telescope]['Summary'].keys())[:-1]  # channels, discard 'Summary'
#print(bands)


# Print Optics information
Print out the relevant fields for each element in the optics chain:  power to detector, efficiency, and cumulative efficiency.

In [6]:
# A function that prints out one field (row) of a table, ie one optical element, 
# but all bands across the columns
def print_optics(exp_dir,field):
    # Load in the two dictionaries that contain all the info we need.
    unpack.unpack_sensitivities(exp_dir)
    unpack.unpack_optical_powers(exp_dir)
    telescope = list(unpack.sens_outputs[run].keys())[0]  # We are working with runs that only have one telescope.
    print(telescope)

    bands = list(unpack.sens_outputs[run][telescope]['Summary'].keys())[:-1]  # channels, discard 'Summary'
    firstch = 'LF_1'
    
    rows = unpack.pwr_outputs[run][telescope][bands[0].split('_')[0]]['Summary'][bands[0]].keys()
    
    str2 = '    '.join(bands)
    print('Band:                    '+str2)
    for row in rows:
        rowstart = row+','
        list2print = [rowstart.ljust(22)]
        for band in bands:
            tube = band.split('_')[0]
            pwrdict = unpack.pwr_outputs[run][telescope][tube]['Summary'][band]
            list2print.append('{0:8.3f},'.format(pwrdict[row][field][0]))
        str2print = ''.join(list2print)
        print(str2print)

 
# Now let's use that function to print out tables for each telescope.
# 3 tables:  power to detector, efficiency, and cumulative efficiency.
for field in ['Power to Detector','Efficiency','Cumulative Efficiency']: 
    print(run)
    print(field)
    print_optics(exp_dir,field)   
    print(" ")


Atacama_CDLAT_FlatBands_20210203
Power to Detector
LAT
Band:                    LF_1    LF_2    MF_1    MF_2    UHF_1    UHF_2
CMB,                     0.034,   0.117,   0.075,   0.111,   0.076,   0.034,
ATM,                     0.068,   0.685,   0.786,   2.452,   8.820,  14.237,
Primary,                 0.003,   0.013,   0.026,   0.111,   0.335,   0.468,
Secondary,               0.003,   0.013,   0.026,   0.111,   0.336,   0.470,
Mirror,                  0.092,   0.375,   0.430,   1.267,   2.548,   2.799,
UHMWWindow,              0.002,   0.014,   0.036,   0.167,   0.487,   0.643,
IRShader1,               0.001,   0.006,   0.015,   0.067,   0.199,   0.258,
IRShader2,               0.001,   0.005,   0.013,   0.056,   0.166,   0.216,
IRShader3,               0.001,   0.004,   0.011,   0.049,   0.146,   0.190,
IRShader4,               0.001,   0.003,   0.009,   0.040,   0.118,   0.153,
IRShader5,               0.000,   0.003,   0.007,   0.033,   0.098,   0.126,
IRShader6,               0

# Print Summary information

In [7]:
# A function that prints out one field (row) of a table, ie one optical element, 
# but all bands across the columns
def print_summary(exp_dir):
    rows = ['Optical Throughput',
            'Optical Power',
            'Psat',
            'Photon NEP',
            'Bolometer NEP',
            'Readout NEP',
            'Total NEP',
            'Detector NET_CMB',
            'Correlation Factor',
            'Array NET_CMB']

    # Load in the two dictionaries that contain all the info we need.
    unpack.unpack_sensitivities(exp_dir)
    telescope = list(unpack.sens_outputs[run].keys())[0]  # We are working with runs that only have one telescope.
    print(telescope)

    bands = list(unpack.sens_outputs[run][telescope]['Summary'].keys())[:-1]  # channels, discard 'Summary'
    
    #rows = unpack.pwr_outputs[run][telescope][bands[0].split('_')[0]]['Summary'][bands[0]].keys()
    
    str2 = '     '.join(bands)
    print('Band:                     '+str2)
    for row in rows:
        rowstart=row+','
        list2print = [rowstart.ljust(22)]
        for band in bands:
            tube = band.split('_')[0]
            if row == 'Psat':
                value = f_safety*unpack.sens_outputs[run][telescope][tube]['Summary'][band]['Optical Power'][0]
            elif (row == 'Total NEP'):
                NEPphonon = unpack.sens_outputs[run][telescope][tube]['Summary'][band]['Bolometer NEP'][0]
                NEPphoton = unpack.sens_outputs[run][telescope][tube]['Summary'][band]['Photon NEP'][0]
                value = np.sqrt(NEPphonon**2 + NEPphoton**2)
            else:
                value = unpack.sens_outputs[run][telescope][tube]['Summary'][band][row][0]
            list2print.append('{0:9.3f},'.format(value))
        str2print = ''.join(list2print)
        print(str2print)

 
# Now let's use that function to print out tables for each telescope.
print(run)
print_summary(exp_dir)

Atacama_CDLAT_FlatBands_20210203
LAT
Band:                     LF_1     LF_2     MF_1     MF_2     UHF_1     UHF_2
Optical Throughput,       0.135,    0.257,    0.174,    0.351,    0.454,    0.446,
Optical Power,            0.240,    1.280,    1.490,    4.650,   13.840,   20.260,
Psat,                     0.720,    3.840,    4.470,   13.950,   41.520,   60.780,
Photon NEP,               4.620,   17.240,   18.300,   44.900,  104.520,  148.480,
Bolometer NEP,            3.830,    8.950,    9.620,   17.020,   29.380,   35.540,
Readout NEP,              1.920,    6.220,    6.620,   15.370,   34.760,   48.880,
Total NEP,                6.001,   19.425,   20.674,   48.018,  108.571,  152.674,
Detector NET_CMB,       287.400,  241.400,  274.500,  310.000,  723.000, 1821.500,
Correlation Factor,       1.240,    1.040,    1.150,    1.010,    1.010,    1.010,
Array NET_CMB,           40.770,   28.710,   11.960,   11.960,   27.660,   69.850,


In [58]:
unpack.sens_outputs[run]['LAT']['LF']

{'Summary': {'LF_1': {'Num Det': [96.0, 0, 0],
   'Optical Throughput': [0.135, 0.0, 0.0],
   'Optical Power': [0.24, 0.0, 0.0],
   'Telescope Temp': [8.44, 0.0, 0.0],
   'Sky Temp': [6.47, 0.0, 0.0],
   'Photon NEP': [4.62, 0.0, 0.0],
   'Bolometer NEP': [3.83, 0.0, 0.0],
   'Readout NEP': [1.92, 0.0, 0.0],
   'Detector NEP': [6.3, 0.0, 0.0],
   'Detector NET_CMB': [287.4, 0.0, 0.0],
   'Detector NET_RJ': [282.4, 0.0, 0.0],
   'Array NET_CMB': [40.77, 0.0, 0.0],
   'Array NET_RJ': [40.06, 0.0, 0.0],
   'Correlation Factor': [1.24, 0.0, 0.0],
   'CMB Map Depth': [67.55, 0.0, 0.0],
   'RJ Map Depth': [66.38, 0.0, 0.0]},
  'LF_2': {'Num Det': [96.0, 0, 0],
   'Optical Throughput': [0.257, 0.0, 0.0],
   'Optical Power': [1.28, 0.0, 0.0],
   'Telescope Temp': [7.73, 0.0, 0.0],
   'Sky Temp': [12.9, 0.0, 0.0],
   'Photon NEP': [17.24, 0.0, 0.0],
   'Bolometer NEP': [8.95, 0.0, 0.0],
   'Readout NEP': [6.22, 0.0, 0.0],
   'Detector NEP': [20.39, 0.0, 0.0],
   'Detector NET_CMB': [241.4, 0.0,

In [10]:
unpack.sens_outputs['Atacama_CDLAT_FlatBands_20210203']['LAT']

{'UHF': {'Summary': {'UHF_1': {'Num Det': [864.0, 0, 0],
    'Optical Throughput': [0.454, 0.0, 0.0],
    'Optical Power': [13.84, 0.0, 0.0],
    'Telescope Temp': [13.58, 0.0, 0.0],
    'Sky Temp': [24.41, 0.0, 0.0],
    'Photon NEP': [104.52, 0.0, 0.0],
    'Bolometer NEP': [29.38, 0.0, 0.0],
    'Readout NEP': [34.76, 0.0, 0.0],
    'Detector NEP': [114.0, 0.0, 0.0],
    'Detector NET_CMB': [723.0, 0.0, 0.0],
    'Detector NET_RJ': [223.3, 0.0, 0.0],
    'Array NET_CMB': [27.66, 0.0, 0.0],
    'Array NET_RJ': [8.54, 0.0, 0.0],
    'Correlation Factor': [1.01, 0.0, 0.0],
    'CMB Map Depth': [45.82, 0.0, 0.0],
    'RJ Map Depth': [14.15, 0.0, 0.0]},
   'UHF_2': {'Num Det': [864.0, 0, 0],
    'Optical Throughput': [0.446, 0.0, 0.0],
    'Optical Power': [20.26, 0.0, 0.0],
    'Telescope Temp': [16.44, 0.0, 0.0],
    'Sky Temp': [39.19, 0.0, 0.0],
    'Photon NEP': [148.48, 0.0, 0.0],
    'Bolometer NEP': [35.54, 0.0, 0.0],
    'Readout NEP': [48.88, 0.0, 0.0],
    'Detector NEP': [160